In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from utils.transformations import FlaggedImputerTransformer
from utils.filters import ZScoreOutlierFilter


### Cargar datos

In [3]:
df = pd.read_csv("data/preprocessed/combined_data.csv")
df.head()


,Price,city,Area,Location,No. of Bedrooms,Resale,MaintenanceStaff,Gymnasium,SwimmingPool,LandscapedGardens,...,LiftAvailable,BED,VaastuCompliant,Microwave,GolfCourse,TV,DiningTable,Sofa,Wardrobe,Stadium
0,8500000,Mumbai,1614,Kandivali West,3,0,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
1,3700000,Mumbai,440,Mira Road East,1,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
2,3500000,Mumbai,890,Kalyan West,2,1,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
3,8400000,Bangalore,3138,Whitefield Hope Farm Junction,4,0,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
4,56000000,Mumbai,2200,Wadala East Wadala,3,1,1,0,0,1,...,1,1,1,0,0,0,0,0,0,0


### Separar features y variable objetivo

In [5]:
X = df.drop(columns=["Price"])
y = df["Price"]


### Aplicar FlaggedImputerTransformer

In [6]:
transformer = FlaggedImputerTransformer()
transformer.fit(X, y)
X_transformed, y_transformed = transformer.transform(X, y)


### Filtrar outliers con ZScoreOutlierFilter

In [7]:
zfilter = ZScoreOutlierFilter(z_thresh=3.0)
X_filtered, y_filtered = zfilter.transform(X_transformed, y_transformed)


### Entrenar modelo con HistGradientBoostingRegressor

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_filtered, test_size=0.2, random_state=42)
model = HistGradientBoostingRegressor()
model.fit(X_train, y_train.ravel())
y_pred = model.predict(X_test)

C:\Users\irene\AppData\Local\Temp\ipykernel_15412\1328206874.py:3: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model.fit(X_train, y_train.ravel())


ValueError: could not convert string to float: 'Mumbai'

### Evaluación del modelo

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAPE: {mape:.2f}%")


### Conclusión

Este segundo experimento aplica el preprocesador `FlaggedImputerTransformer`, que realiza:
- Imputación con la mediana
- Creación de indicadores de missing
- Expansión no lineal con raíz cuadrada de 'Area'

Junto con `ZScoreOutlierFilter` y el modelo `HistGradientBoostingRegressor`, permite comparar si esta estrategia funciona mejor o peor que la anterior.
